In [3]:
from datetime import datetime, timedelta
from sqlalchemy import create_engine, MetaData, Table
from scrape_sofa import get_sofascore_api_data, init_driver, get_all_players_stats_fbref, get_all_teams_stats_fbref
from funciones import get_stat
import pandas as pd

today = datetime.now()
yesterday = today - timedelta(days=1)
yesterday = yesterday.strftime('%Y-%m-%d')
engine = create_engine('mysql+pymysql://root@localhost/footviz')
df_partidos_ayer = pd.read_sql(f"SELECT * FROM partido WHERE fecha = '{yesterday}'", engine)


In [4]:

df_alineaciones = pd.read_sql(f'SELECT id_alineacion FROM alineaciones WHERE partido = 12', engine)

In [7]:
alineacion = int(df_alineaciones['id_alineacion'])
alineacion

12